In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from warnings import filterwarnings as filt
import cv2
import os
from sklearn.model_selection import train_test_split
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import random
from sklearn.model_selection import train_test_split

In [4]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Data Loading
base_path = "/content/drive/MyDrive/Machine Learning/melanoma_cancer_dataset/"
train_path = os.path.join(base_path, 'train')
test_path  = os.path.join(base_path, 'test')

In [ ]:
def dataset(path):
    data = {
        'img_path' : [],
        'target_class'   : [],
    }
    for class_name in os.listdir(path):
        ip = [os.path.join(class_name, i) for i in os.listdir(os.path.join(path, class_name))]
        data['img_path'] += ip
        data['target_class']   += [class_name] * len(ip)

    print(f'Total no of images : {len(data["img_path"])}')
    return data

In [ ]:
data = dataset(train_path)
df = pd.DataFrame(data)

In [ ]:
df.head()

In [ ]:
df.count()

In [ ]:
train = ImageFolder(train_path, transform=transforms.ToTensor())
train[0][1]

In [ ]:
def show_images(train, num_imgs, col):
    row = 0;
    if num_imgs % col == 0:
        row = num_imgs // col
    else:
        row = (num_imgs // col) + 1

    plt.figure(figsize = (col * 4, row * 4))

    for i in range(num_imgs):
        img = train[random.randint(0,9000)][0].permute(1, 2, 0)
        plt.subplot(row, col, i + 1)

        plt.imshow(img)
        plt.axis(False)

    plt.show()

In [ ]:
# Images before preprocessing
show_images(train, 5, 5)

In [ ]:
import albumentations as alb
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# data augmentation techniques in transforms
def transforms(image, normalize = True):
    transform = alb.Compose([
        alb.OneOf([
            alb.Rotate(limit = 60, border_mode = cv2.BORDER_CONSTANT),
            alb.Rotate(limit = 100, border_mode = cv2.BORDER_CONSTANT),
        ], p = 1),
        alb.HorizontalFlip(), alb.VerticalFlip(),
        alb.CenterCrop(250 ,250, p = 0.4),
        alb.GaussianBlur(p = 0.5),alb.Resize(250, 250)
    ])
    x = transform(image = image)['image']
    if normalize:
        x = x / 255
    return x

In [ ]:
def create_image_data_generator(df, train_path, x, y, data_gen_args = {}, **kwargs):
    imagegen = ImageDataGenerator(**data_gen_args)
    data_loader = imagegen.flow_from_dataframe(df, train_path, x_col = x, y_col = y, **kwargs)
    return data_loader

In [ ]:
def sample(x, y, frac = 0.2, return_val = True):
    x,xt, y, yt = train_test_split(x, y, test_size = frac, stratify = y)
    train = pd.concat([x, y], axis = 1)
    val = pd.concat([xt, yt], axis = 1)
    if return_val:
        return train, val
    return train

In [ ]:
x = df.drop(['target_class'], axis = 1)
y = df.target_class

train_df, val_df = sample(x, y, 0.15)
train_df.shape

In [ ]:
# creating a data loader
t_dargs = {
    'preprocessing_function' : transforms
}

v_dargs = {
    'preprocessing_function' : lambda x : x / 255
}

train_loader = create_image_data_generator(train_df, train_path, 'img_path', 'target_class', t_dargs, batch_size = 64,
                               color_mode = 'rgb', class_mode = 'categorical', target_size = (250, 250), shuffle = True)
val_loader = create_image_data_generator(val_df, train_path, 'img_path', 'target_class', v_dargs, batch_size = 64,
                               color_mode = 'rgb', class_mode = 'categorical', target_size = (300, 300), shuffle = False)

In [ ]:
type(train_loader)

In [ ]:
type(train_loader)

In [ ]:
#method that preprocessed images
def show_imgs_from_loader(loader, n_img, col):
    n_img = min(loader[0][0].shape[0], n_img)
    row = 0;
    if n_img % col == 0:
        row = n_img // col
    else:
        row = (n_img // col) + 1

    images, labels = loader[0]

    plt.figure(figsize = (col * 4, row * 4))

    for i in range(n_img):
        image = images[i]
        label = labels[i]
        plt.subplot(row, col, i + 1)

        plt.imshow(image)
        plt.axis(False)

    plt.show()

In [ ]:
# displaying images after preprocessing
show_imgs_from_loader(train_loader, 5, 5)

# Phase 3 - Building the model

In [ ]:
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, MaxPool2D, Flatten

In [ ]:
# building the model
cnn_vgg_model = VGG16(weights = 'imagenet', include_top = False, input_shape = (224,224,3))

for layer in cnn_vgg_model.layers[3:]:
    layer.trainable = False

output = Flatten()(cnn_vgg_model.output)
output = Dense(1024, activation = 'relu')(output)
output = Dense(512,  activation = 'relu')(output)
output = Dense(2, activation = 'softmax')(output)

cnn_vgg_model = Model(inputs = [cnn_vgg_model.input], outputs = [output])

In [ ]:
cnn_vgg_model.summary()

In [ ]:
# Callbacks to stramline the training process.
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', patience = 2, mode = 'min', verbose = 1)
checkpoint = ModelCheckpoint('cnn-model.h5', monitor = 'val_loss', verbose = 1, save_best_only = True, mode = 'min')
early_stop = EarlyStopping(monitor = 'val_loss', patience = 4, verbose = 1)

In [ ]:
# Optimizers and Lossfunctions
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, SGD, Adagrad

cnn_vgg_model.compile(optimizer = Adagrad(learning_rate = 0.01),
              loss = CategoricalCrossentropy(),
              metrics = ['accuracy'])

In [ ]:
train_loader.batch_size

In [ ]:
# training the model
epoch = 1
cnn_vgg_model.fit(
    train_loader,
    steps_per_epoch = np.ceil(train_df.shape[0] / train_loader.batch_size),
    epochs = epoch,
    callbacks = [reduce_lr, checkpoint, early_stop],
    validation_data = val_loader,
    shuffle = True
    )